In [1]:
!pip install rioxarray
!pip install geopandas
!pip install tqdm
!pip install google.cloud
!pip install configparser
!pip install optuna
!pip install catboost
!pip install pickle5
!pip install shap
!pip install geocube
!pip install fsspec
!pip install gcsfs

     |████████████████████████████████| 46 kB 4.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 19.3 MB 1.4 MB/s 
     |████████████████████████████████| 6.3 MB 19.5 MB/s 
  Created wheel for rioxarray: filename=rioxarray-0.7.1-py3-none-any.whl size=54118 sha256=d1c56a8813a0117e3de2b912c5d3ebb54bcbfb5649932771829fd41e494e87fc
  Stored in directory: /root/.cache/pip/wheels/42/9b/1c/acf2904f7ee44be3c0b8ffc7ea4fafebb135aee00242bef86f
Successfully built rioxarray
     |████████████████████████████████| 1.0 MB 29.8 MB/s 
     |████████████████████████████████| 15.4 MB 40 kB/s 
     |████████████████████████████████| 308 kB 30.6 MB/s 
     |████████████████████████████████| 209 kB 37.8 MB/s 
     |████████████████████████████████| 80 kB 10.7 MB/s 
     |████████████████████████████████| 75 kB 5.0 MB/s 
     |████████████████████████████████| 49 kB 5.3 MB/s 
     |████

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/03 PEMETAAN/01 MODEL - DRIVE

import os
# os.listdir()

/content/drive/MyDrive/03 PEMETAAN/01 MODEL - DRIVE


In [4]:
import pandas as pd
import numpy as np
import os
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import configparser
config = configparser.ConfigParser()
config.read('config/config_ecoregion_3001.ini')
folder=config['default']['folder']
train_data=config['output']['train_data']

## Load Data

In [5]:
file_name='kerangka-spasial/ml_learning/train_data/'+train_data
data=pd.read_csv(file_name,sep=';')
data.head()
data.TRAIN_CLASS=data.TRAIN_CLASS.astype(str)

In [6]:
data.TRAIN_CLASS

0         7
1         7
2         7
3         7
4         7
         ..
78229    12
78230    12
78231    12
78232    12
78233    12
Name: TRAIN_CLASS, Length: 78234, dtype: object

## Sample Size
This chunk would show the size of training sample groupped by training class.

In [7]:
pivot_=data.pivot_table(index='TRAIN_CLASS',values='x',aggfunc='count').reset_index().rename(columns={'x':'SAMPLE SIZE'})
pivot_['TRAIN_CLASS']=pivot_['TRAIN_CLASS'].astype(str)
fig = px.bar(pivot_, x="TRAIN_CLASS", y="SAMPLE SIZE", title="Sample Size")
fig.show()

Hover the bar to know the size of sample.

## Interactive Histogram

This chunk would show the histogram that could be shown interactively. Please consider bellowing steps:
<ol>
    <li>Change the tc_show into your desired class
    </li>
    <li>Change the ls_columns_hist into desired features to be showed.
    </li>
    <li>Don't be detailed or specify more than 4 class/features. It would crash the code
    </li>
</ol>

In [8]:
fig=go.Figure()
btn_1=[]
tc_show=['1','2','4']
ls_columns_hist=['S2_B11med', 'S2_B8med']
for c_ in ls_columns_hist:
    if c_=='S2_B11med':
        for k_ in tc_show:
            fig.add_trace(go.Histogram(
                    x=data.loc[data.TRAIN_CLASS==k_,c_],
                    y=data['x'],
                    name=k_,
                    nbinsx=100,
                    visible=True))
    
    else:
        for k_ in tc_show:
            fig.add_trace(go.Histogram(
                    x=data.loc[data.TRAIN_CLASS==k_,c_],
                    y=data['x'],
                    name=k_,
                    nbinsx=100,
                    visible=False))
    btn_1.append(dict(
        label=c_,
        method="update",
        args=[{"visible": [c_==col for col in ls_columns_hist]},
        ]))

btn_list_1 = list(btn_1)


fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.75)
fig.update_layout(
    updatemenus=[
        dict(buttons=btn_list_1)
    ])

fig.show()

Output hidden; open in https://colab.research.google.com to view.

## Kernel Density Plot

Compared with previous plot, if you want to compare one class with others in specific features. Run this code. Change the value of <code>columns_show</code> into the feature that you wanna look.

In [9]:
hist_data=[]
train_data=[]
columns_show='S2_B2mean'
for i in data.TRAIN_CLASS.unique():
    hist_data.append(data.loc[data.TRAIN_CLASS==i].dropna()[columns_show].values)
    train_data.append(i)

fig = ff.create_distplot(hist_data, train_data, show_hist=False, show_rug=False )

# Add title
fig.update_layout(title_text='Kernel Density Plot Comparison for '+columns_show)
fig.show()

## Radar Plot

This code would show the radar plot to compare the average value of each selected feature group of your class.

In [10]:
spectral_med=data[['S2_B12med', 'S2_B11med', 'S2_B8med', 'S2_B4med', 'S2_B3med',
       'S2_B2med','TRAIN_CLASS']].pivot_table(index='TRAIN_CLASS',values=['S2_B12med', 'S2_B11med', 
                                                            'S2_B8med', 'S2_B4med', 'S2_B3med','S2_B2med'],
                               aggfunc='mean')
spectral_means=data[['S2_B12mean','S2_B11mean', 'S2_B8mean', 'S2_B4mean', 'S2_B3mean', 
                     'S2_B2mean','TRAIN_CLASS']].pivot_table(index='TRAIN_CLASS',values=[ 'S2_B12mean',
                                            'S2_B11mean', 'S2_B8mean', 'S2_B4mean', 'S2_B3mean', 'S2_B2mean',],
                               aggfunc='mean')
indices_med=data[[ 'ARVI_p50', 'SAVI_p50', 'NDBI_p50',
       'mNDWI_p50', 'NDWI_p50', 'mNDVI_p50', 'NDVI_p50','TRAIN_CLASS']].pivot_table(index='TRAIN_CLASS',
        values=[ 'ARVI_p50', 'SAVI_p50', 'NDBI_p50',
       'mNDWI_p50', 'NDWI_p50', 'mNDVI_p50', 'NDVI_p50'],
                               aggfunc='mean')
indices_means=data[['ARVI_mean', 'SAVI_mean', 'NDBI_mean',
       'mNDWI_mean', 'NDWI_mean', 'mNDVI_mean', 'NDVI_mean','TRAIN_CLASS']].pivot_table(index='TRAIN_CLASS',
        values=[ 'ARVI_mean', 'SAVI_mean', 'NDBI_mean',
       'mNDWI_mean', 'NDWI_mean', 'mNDVI_mean', 'NDVI_mean'],aggfunc='mean')

Do change the <code>rdr_data</code> into desired plotted data. For example: if you want to plot the comparison between medians indices over training class, do change the code into <code>rdr_data=indices_med</code>

In [11]:
rdr_data=spectral_med

In [12]:
fig=go.Figure()
for i in rdr_data.index.values.tolist():
    r_data=rdr_data.iloc[rdr_data.index==i,].values.tolist()[0]
    fig.add_trace(go.Scatterpolar(
        r=r_data,
        theta=rdr_data.columns.tolist(),
        fill='tonext',
        name=i))
fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[min(rdr_data.min()), max(rdr_data.max())]
    )),
  showlegend=True
)
fig.update_layout(title_text='Radar Plot')
fig.show()

## Correlation Heatmap

This chunks would show the correation heatmaps in every train class.

In [13]:
fig=go.Figure()
tr_cl=data.TRAIN_CLASS.unique()
btn_1=[]
cr_col=['wet_mean','green_mean', 'bright_mean', 'ARVI_mean', 'SAVI_mean', 'NDBI_mean',
       'mNDWI_mean', 'NDWI_mean', 'mNDVI_mean', 'NDVI_mean', 'wet_p50',
       'green_p50', 'bright_p50', 'ARVI_p50', 'SAVI_p50', 'NDBI_p50',
       'mNDWI_p50', 'NDWI_p50', 'mNDVI_p50', 'NDVI_p50', 'S2_B12mean',
       'S2_B11mean', 'S2_B8mean', 'S2_B4mean', 'S2_B3mean', 'S2_B2mean',
       'S2_B12med', 'S2_B11med', 'S2_B8med', 'S2_B4med', 'S2_B3med',
       'S2_B2med']
for i in tr_cl:
    corr=data.loc[data.TRAIN_CLASS==i,cr_col].corr().T.values
    if i=='1':
        fig.add_trace(go.Heatmap(z=corr,
                  x=cr_col,
                  y=cr_col,
                  xgap=1, ygap=1,visible=True))
    else:
        fig.add_trace(go.Heatmap(z=corr,
                  x=cr_col,
                  y=cr_col,
                  xgap=1, ygap=1,visible=False))
    btn_1.append(dict(
        label=i,
        method="update",
        args=[{"visible": [i==cl for cl in tr_cl]},
        ]))
btn_list_1 = list(btn_1)

fig.update_layout(
    updatemenus=[
        dict(buttons=btn_list_1)
    ])

fig.show()

## BoxPlot Outlier Check

This chunks would show the boxplot for outlier identification. Same as histogram, please consider following steps:
<ol>
    <li>Specify the class in <code>tc_show</code></li>
    <li>Specify the feature in <code>ls_columns_hist</code></li>
    <li>Don't be DETAILED... if you put more than 4 class or features, it would slow down the process or maybe head into crash.</li>
</ol>

In [14]:
fig=go.Figure()
btn_b=[]
tc_show=['1','2','4','34']
ls_columns_hist=['S2_B11med', 'S2_B8med', 'S2_B2med']
for c_ in ls_columns_hist:
    if c_=='S2_B11med':
        for k_ in tc_show:
            fig.add_trace(go.Box(
                    y=data.loc[data.TRAIN_CLASS==k_,c_],
                    name=k_, boxpoints='outliers',
                    visible=True))
    
    else:
        for k_ in tc_show:
            fig.add_trace(go.Box(
                    y=data.loc[data.TRAIN_CLASS==k_,c_],
                    name=k_, boxpoints='outliers',
                    visible=False))
    btn_b.append(dict(
        label=c_,
        method="update",
        args=[{"visible": [c_==col for col in ls_columns_hist]},
        ]))

btn_list_1 = list(btn_b)

fig.update_layout(
    updatemenus=[
        dict(buttons=btn_list_1)
    ])

fig.show()